In [5]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [6]:
ORS_API_KEY = os.getenv("ORS_API_KEY")

In [7]:
import openrouteservice
from openrouteservice import convert

def get_route_and_distance(start_coords, end_coords, api_key):
  client = openrouteservice.Client(key=api_key)  # Use your API key here
  
  # Define the start and end points
  coords = (start_coords, end_coords)
  
  # Request directions
  routes = client.directions(coords)
  
  # Extract distance and duration
  distance = routes['routes'][0]['summary']['distance'] / 1000  # Convert meters to kilometers
  duration = routes['routes'][0]['summary']['duration'] / 3600  # Convert seconds to hours
  
  print(f"Distance: {distance:.2f} km")
  print(f"Estimated Duration: {duration:.2f} hours")
  
  # For detailed route steps, uncomment the following lines:
  # for step in routes['routes'][0]['segments'][0]['steps']:
  #     print(step['instruction'])

# Example usage
api_key = ORS_API_KEY
start_coords = (-73.985428, 40.748817)  # Longitude, Latitude for New York (Empire State Building)
end_coords = (-118.405128, 34.076214)  # Longitude, Latitude for Los Angeles (Los Angeles City Hall)
get_route_and_distance(start_coords, end_coords, api_key)

Distance: 4511.70 km
Estimated Duration: 45.03 hours
